In [ ]:
import os
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv
import numpy as np
from redfin_scraping_utils import RentScraper, BuyScraper
from geocoding_utils import Geocoder 
import geopandas as  gpd
from rentprediction_utils import RentPredictor as RP1  # Replace with your actual module names


In [2]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

def clean_data_for_write(df):
    """
    Cleans the real estate dataset by:
    - Removing duplicate columns
    - Converting column names to lowercase
    - Ensuring appropriate data types
    - Replacing NaN values and 'NaN' strings with None for JSON compatibility
    
    Args:
        df (pd.DataFrame): The DataFrame to clean.
    
    Returns:
        pd.DataFrame: The cleaned DataFrame ready for writing to Supabase.
    """
    # Print initial number of rows
    print(f"Initial number of rows: {len(df)}")

    # Convert all column names to lowercase
    df.columns = [col.lower() for col in df.columns]
    print("Converted all column names to lowercase.")

    # Remove duplicate columns, keeping the first occurrence
    duplicate_columns = df.columns[df.columns.duplicated()].tolist()
    if duplicate_columns:
        df = df.loc[:, ~df.columns.duplicated()]
        print(f"Removed duplicate columns: {duplicate_columns}")
    else:
        print("No duplicate columns found.")

    # Define a mapping of columns to their desired data types
    # Adjust this mapping based on your data's specifics
    dtype_mapping = {
        'property_id': 'Int64',
        'listing_id': 'Int64',
        'mls_id': 'string',
        'status': 'string',
        'price': 'Int64',
        'hoa_fee': 'float64',
        'square_feet': 'float64',
        'lot_size': 'float64',
        'bedrooms': 'Int64',
        'bathrooms': 'float64',
        'location': 'string',
        'stories': 'float64',
        'address': 'string',
        'city': 'string',
        'state': 'string',
        'zip_code': 'string',
        'year_built': 'Int64',
        'url': 'string',
        'latitude': 'float64',
        'longitude': 'float64',
        'description': 'string',
        'property_type': 'Int64',
        'country_code': 'string',
        'cbg_geoid': 'string',
        'cbsa_geoid': 'string',
        'cbsa_name': 'string',
        'state_id': 'Int64',
        'state_code': 'string',
        'beds_rent_benchmark_5_neighbors': 'float64',
        'beds_average_distance_5_neighbors': 'float64',
        'beds_rent_benchmark_10_neighbors': 'float64',
        'beds_average_distance_10_neighbors': 'float64',
        'state_code_average_rent': 'float64',
        'cbsa_geoid_average_rent': 'float64',
        'predicted_rent': 'float32'
    }

    # Convert columns to appropriate data types
    for col, dtype in dtype_mapping.items():
        if col in df.columns:
            try:
                if dtype.startswith('Int'):
                    # For integer columns with nullable type
                    df[col] = pd.to_numeric(df[col], errors='coerce').astype(dtype)
                elif dtype in ['float64', 'float32']:
                    df[col] = pd.to_numeric(df[col], errors='coerce').astype(dtype)
                elif dtype in ['category', 'string']:
                    df[col] = df[col].astype(dtype)
                else:
                    df[col] = df[col].astype(dtype)
            except Exception as e:
                print(f"Error converting '{col}' to {dtype}: {e}")
        else:
            print(f"Column '{col}' not found in DataFrame.")

    # Replace all pandas NaN with None
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(0)  # Fill NaN with 0 for numeric columns
        elif pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].cat.fillna('Unknown')  # Add 'Unknown' as category and fill NaN
    
    print("Replaced all pandas NaN values with 0 or Unkown.")



    # Final check for remaining NaNs
    total_nans = df.isna().sum().sum()
    if total_nans == 0:
        print("All NaNs have been successfully replaced with None.")
    else:
        print(f"Warning: There are still {total_nans} NaNs remaining.")
        # Identify columns with NaNs
        nan_columns = df.columns[df.isna().any()].tolist()
        print(f"Columns with NaNs: {nan_columns}")

    # Print final number of rows
    print(f"Final number of rows: {len(df)}")
    
    return df



In [14]:
#parameters
states = ['WA', 'ID', 'OR', 'MI', 'IL', 'IA', 'WI', 'MN', 'IN']

basic_features = ["square_feet", "bedrooms", "bathrooms"]
basic_metadata = ['mls_id', 'status', 'price', 'hoa_fee', 'lot_size', 
       'location', 'stories', 'address', 'city', 'state', 'zip_code',
       'year_built', 'url', 'latitude', 'longitude']

max_bedrooms = 6
min_bedrooms = 0
max_bathrooms = 4
min_bathrooms = 0
max_sqft = 5000
max_rent = 10000

rent_predictor = RP1(
    knn_model_path=r'..\Models&Training\knn_beds_model.pkl',
    xgboost_model_path=r'..\Models&Training\xgboost_rent_prediction_model.pkl',
    rent_benchmarks_path=r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\rentals_0926.csv",
    n_values=[5, 10],
    min_bedrooms=min_bedrooms,
    max_bedrooms=max_bedrooms,
    max_bathrooms=max_bathrooms,
    max_rent=max_rent,
    basic_features=['square_feet', 'bathrooms', 'bedrooms', 
                    'latitude', 'longitude', 'state_code', 'cbsa_geoid'],
    verbose=True

)



KNN model loaded from ..\Models&Training\knn_beds_model.pkl.
XGBoost model loaded from ..\Models&Training\xgboost_rent_prediction_model.pkl.
Using provided basic_features.
Loading rent benchmarks from C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\rentals_0926.csv...
Initial number of rows: 45209
Renamed columns for consistency.
Converted 'square_feet' to numeric. NaNs before: 11899, after: 11899
Converted 'bathrooms' to numeric. NaNs before: 0, after: 0
Converted 'bedrooms' to numeric. NaNs before: 0, after: 0
Converted 'latitude' to numeric. NaNs before: 0, after: 0
Converted 'longitude' to numeric. NaNs before: 0, after: 0
Converted 'rent' to numeric. NaNs before: 81, after: 81
Cleaned 'state_code' column: converted to uppercase and stripped whitespace.
Cleaned 'cbsa_geoid' column: stripped whitespace.
All essential columns are present.
Dropped duplicate property IDs. Rows before: 45209, after: 45209
Unique property types in the data: [ 6  5 13  4]
Applying filter

In [4]:
load_dotenv()

# Create Supabase Client
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

#Initialize Scraper
buy_scraper = BuyScraper()
rent_scraper = RentScraper()

In [5]:
states = ['ID']
for state in states:
    try:
        # Initialize a new DataFrame for each state
        buy_df = pd.DataFrame()

        # Scrape for-sale listings and append to buy_df
        buy_data = BuyScraper().scrape_state(state)  # Ensure BuyScraper is properly imported and initialized

        if not buy_data.empty:
            buy_df = pd.concat([buy_df, buy_data], ignore_index=True)

        # Replace NaN with None and drop duplicates based on 'property_id'
        buy_df = buy_df.replace(np.nan, None)
        buy_df = buy_df.drop_duplicates(subset="property_id")

        records = buy_df.to_dict(orient='records')

        print(f"Scraped {len(buy_data)} for-sale listings for state {state}")

        # Initialize Geocoder and perform geocoding
        geocoder = Geocoder(
            buy_df, 
            latitude_col='latitude', 
            longitude_col='longitude'
        )

        print(f"Geocoding {state}")
        df_geocoded = geocoder.geocode_all(
            demographic_areas_path=r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\census_block_group_source_nationwide\v107\blkgrp.gdb",
            cbsa_source_path=r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\cbsa_source\tl_2020_us_cbsa.shp", 
            state_source_path=r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Areas\state_source\States_shapefile.shp"
        )

        print(f"Predicting Rent in {state}")
        # Process the geocoded data to predict rent
        df_predicted = rent_predictor.process(df_geocoded, state)

        df_clean = clean_data_for_write(df_predicted)

        print(f"Writing {state}")

        response = (
            supabase.table("redfin_listings_bronze")
            .upsert(df_clean.to_dict(orient="records"), on_conflict="property_id")
            .execute()
        )

        print(f"Write Complete for state {state}")

    except Exception as e:
        print(f"An error occurred while processing state {state}: {e}")

Scraping 325 Zip Codes in ID
Processing 24% done (81/325 zip codes)
Processing 49% done (162/325 zip codes)
Processing 74% done (243/325 zip codes)
Scraped 16406 for-sale listings for state ID
Geocoding ID
Predicting Rent in ID
An error occurred while processing state ID: 'state_code'


In [10]:
print(f"Predicting Rent in {state}")
# Process the geocoded data to predict rent
df_predicted = rent_predictor.process(df_geocoded, state)

df_clean = clean_data_for_write(df_predicted)

print(f"Writing {state}")


Predicting Rent in ID


KeyError: 'state_code'

In [16]:
df_rent = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\rentals_0926.csv")

property_types_to_include = ["6"]  # Modify based on actual data
filters = (
    (df_rent["bedrooms"] > min_bedrooms) &
    (df_rent["bedrooms"] < max_bedrooms) &
    (df_rent["bathrooms"] < max_bathrooms) &
    (df_rent["state_code"].notna()) &
    (df_rent["property_type"].isin(property_types_to_include)) &  # Adjusted property types
    (df_rent["rent"] <= max_rent)
)

print(f"Number of rows before filtering: {len(df_rent)}")
df_filtered = df_rent[filters]

print(f"Number of rows after filtering: {len(df_filtered)}")


KeyError: 'rent'